In [6]:
SELECT TOP (5) *
FROM [dbo].[Customer]

(5 rows affected)

Total execution time: 00:00:00.054

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [60]:
/*Query total orders, total sales and total payments*/
SELECT COUNT(a.order_id) AS Total_Orders,
      ROUND(SUM(b.price + b.freight_value),2) AS Total_Sales,
      ROUND(SUM(payment_value),2) AS Total_payments 
      FROM [dbo].[orders] AS a 
      INNER JOIN [dbo].[orderitems] AS b 
      ON a.order_id = b.order_id
      INNER JOIN orderpayments AS c 
      ON a.order_id = c.order_id

(1 row affected)

Total execution time: 00:00:02.831

Total_Orders,Total_Sales,Total_payments
117601,16566543.85,20308134.71


In [66]:
/* Query number orders got review 5 score and percentage of it*/
SELECT SUM(v.review_5) AS order_review_5,
       COUNT(v.total_orders) AS order_total,
       ROUND(100*SUM(v.review_5)/COUNT(v.total_orders),0) AS percentage_rev_5
FROM
(SELECT CASE WHEN review_score = 5 THEN 1 ELSE 0 END AS review_5,
       CASE WHEN review_score < 6 THEN 1 ELSE NULL END AS total_orders  
FROM orderreviews) AS v

(1 row affected)

Total execution time: 00:00:00.085

order_review_5,order_total,percentage_rev_5
57328,99224,57


In [118]:
/*Total sales and total orders by month in each year 2016, 2017, 2018*/
WITH cte_sales AS (SELECT FORMAT(order_approved_at, 'yyyy-MM') AS date_order,
       ROUND(SUM(price+freight_value),2) as Total_sales,
       YEAR(order_approved_at) as year,
       MONTH(order_approved_at) as month,
       COUNT(CASE WHEN order_approved_at LIKE '201%' THEN 1 ELSE NULL END) AS Total_orders
FROM orders AS t 
INNER JOIN orderitems as u 
ON t.order_id = u.order_id
GROUP BY YEAR(order_approved_at), MONTH(order_approved_at), 
         FORMAT(order_approved_at, 'yyyy-MM')
)

SELECT date_order, Total_sales, Total_orders 
FROM cte_sales
ORDER BY date_order ASC


Warning: Null value is eliminated by an aggregate or other SET operation.

(25 rows affected)

Total execution time: 00:00:19.099

date_order,Total_sales,Total_orders
NULL,1954.6,0
2016-09,143.46,3
2016-10,57020.13,366
2016-12,19.62,1
2017-01,130840.55,908
2017-02,286135.92,1931
2017-03,428557.47,3022
2017-04,409570.85,2655
2017-05,587151.63,4130
2017-06,507643.19,3602


In [81]:
/* Total sales, average sales, number of orders, total payments and average payments by customer state and city*/
WITH temp_customer AS (SELECT t.price, t.order_id, s.customer_id, t.payment_value, customer_city, customer_state 
FROM 
(SELECT v.order_id, v.price, u.customer_id, p.payment_value
FROM orders AS u
INNER JOIN orderitems AS v 
ON u.order_id = v.order_id
INNER JOIN orderpayments as p 
ON u.order_id = p.order_id) AS t 
INNER JOIN Customer as s 
ON s.customer_id = t.customer_id)

SELECT customer_state,
       customer_city,
       ROUND(AVG(price), 2) AS avg_sales, 
       ROUND(SUM(price),2) AS total_sales,
       COUNT(order_id) AS total_orders,
       ROUND(AVG(payment_value),2) AS avg_customer_payments,
       ROUND(SUM(payment_value),2) AS total_customer_payments
FROM temp_customer
GROUP BY customer_state, customer_city
ORDER BY total_orders DESC


(5 rows affected)

Total execution time: 00:00:06.186

customer_state,customer_city,avg_sales,total_sales,total_orders,avg_customer_payments,total_customer_payments
SP,sao paulo,108.03,2008254.39,18590,152.77,2839903.47
RJ,rio de janeiro,125.67,1030715.84,8202,191,1566591.49
MG,belo horizonte,113.02,366981.33,3247,154.38,501261.5
DF,brasilia,127.1,312274.57,2457,175.21,430499.43
PR,curitiba,120.8,218532.47,1809,182.05,329321.54


In [103]:
/* Performance of seller on total number orders, total number of customers, total profit, total delivered orders and number of orders with 5 score review */
WITH temp_seller AS (SELECT o.order_id, c.customer_id, 
       customer_unique_id, 
       customer_city, customer_state, i.price, i.freight_value,
       s.seller_id, seller_city, seller_state,
       order_status, review_score
FROM orders AS o
INNER JOIN Customer AS c 
ON o.customer_id = c.customer_id
INNER JOIN orderitems AS i 
ON o.order_id = i.order_id 
INNER JOIN sellers AS s 
ON i.seller_id = s.seller_id
INNER JOIN orderreviews AS r
ON o.order_id = r.order_id)

SELECT TOP(5) seller_id, COUNT(order_id) AS total_orders,
       COUNT(customer_unique_id) AS total_customers,
       ROUND(SUM(price-freight_value),2) AS profit,
       SUM(CASE WHEN order_status = 'delivered' THEN 1 ELSE 0 END) AS total_delivered_orders,
       SUM(CASE WHEN review_score > 4 THEN 1 ELSE 0 END) AS total_review5_score
FROM temp_seller
GROUP BY seller_id
ORDER BY total_orders DESC




(5 rows affected)

Total execution time: 00:00:06.821

seller_id,total_orders,total_customers,profit,total_delivered_orders,total_review5_score
6560211a19b47992c3666cc44a7e94c0,2020,2020,94658.35,1985,1024
4a3ca9315b744ce9f8e9374361493884,1984,1984,165541.16,1949,947
1f50f920176fa81dab994f9023523100,1932,1932,71849.25,1927,1096
cc419e0650a3c5ba77189a1882b7556a,1811,1811,79840.25,1755,1053
da8622b14eb17ae2831f4ac5b9dab84a,1568,1568,136787.37,1565,893


In [117]:
/* Total profit and total orders of each product category */
SELECT product_category_name,
       ROUND(SUM(price-freight_value),2) AS total_profits,
       ROUND(COUNT(order_id),2) AS total_orders
FROM
(SELECT i.product_id, order_id, price, 
       freight_value, product_category_name
FROM orderitems AS i 
INNER JOIN products AS p 
ON i.product_id = p.product_id) AS p
GROUP BY product_category_name
ORDER BY total_profits DESC

(74 rows affected)

Total execution time: 00:00:01.689

product_category_name,total_profits,total_orders
relogios_presentes,1104469.75,5991
beleza_saude,1076114.61,9670
cama_mesa_banho,832295.64,11115
esporte_lazer,819441.46,8641
informatica_acessorios,764636.24,7827
moveis_decoracao,557013.19,8334
cool_stuff,551251.75,3796
automotivo,500055.9,4235
utilidades_domesticas,486099.55,6964
brinquedos,406520.65,4117


In [129]:
/* Top 5 city with the largest order amount in each year */
WITH Temp_rank AS
(SELECT Year, customer_id, customer_city,
        order_amount,
       DENSE_RANK() OVER(PARTITION BY Year ORDER BY order_amount DESC) AS Rank
FROM
(SELECT c.customer_id, customer_city,
       o.order_id, payment_value AS order_amount,
       FORMAT(order_approved_at, 'yyyy') AS Year
FROM Customer AS c 
INNER JOIN orders AS o 
ON c.customer_id = o.customer_id
INNER JOIN orderpayments as p 
ON o.order_id = p.order_id) AS d 
WHERE Year IS NOT NULL)

SELECT * FROM Temp_rank WHERE Rank < 6

(15 rows affected)

Total execution time: 00:00:27.373

Year,customer_id,customer_city,order_amount,Rank
2016,a9dc96b027d1252bbac0a9b72d837fc6,rio de janeiro,1423.55,1
2016,1d34ed25963d5aae4cf3d7f3a4cda173,quissama,1400.74,2
2016,4a06381959b6670756de02e07b83815f,vargem grande paulista,1227.78,3
2016,1fc56719b52f82c03caddc5faf531fbb,santo antonio da patrulha,982.41,4
2016,85f0e92957e9fb9c5f72ba5378f492a0,rio de janeiro,979.91,5
2017,1617b1357756262bfa56ab541c47bc16,rio de janeiro,13664.08,1
2017,c6e2731c5b391845f6800c97401a43a9,campo grande,6929.31,2
2017,3fd6777bbce08a352fddd04e4a7cc8f6,marilia,6726.66,3
2017,05455dfa7cd02f13d132aa7a6a9729c6,divinopolis,6081.54,4
2017,df55c14d1476a9a3467f131269c2477f,araruama,4950.34,5
